In [2]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt 
from IPython.display import display
from tqdm import tqdm_notebook
import tqdm
import seaborn as sns
from choices import param_dict
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import  IterativeImputer 
from sklearn.impute import SimpleImputer , KNNImputer
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import GradientBoostingClassifier
sns.set(font_scale=1.3, rc={'axes.facecolor':'white', 'figure.facecolor':'white' , 'axes.grid':True} , style="whitegrid")
# 
%load_ext autoreload
%autoreload 2
import time 
tstart = time.time()
feat_to_drop = param_dict['hardness']+param_dict['IRAC']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:

from utilities import deets
from choices import get_train_data , param_dict
classes = ['AGN' ,'STAR' , 'YSO' ,  'CV' , 'LMXB' , 'HMXB' ,'ULX','PULSAR']
flag = {
    'conf_flag' : 0 , 
    'streak_src_flag' : 0 , 
    'extent_flag' : 0 , 
    'pileup_flag' : 0 , 
    }
ret_dict= {
    'clf': False,
 'prob_table': True,
 'acc': True,
 'pr_score': True,
 'precision': True,
 'recall': True , 
 'roc_auc_score' : False
 }



In [4]:

file = f'../compiled_data_v3/imputed_data_v2/x_phot_minmax_modeimp.csv'
data = get_train_data(flags = flag, classes= classes , offset = 1, file=file)
data = data.drop(columns = feat_to_drop)
deets(data,0)


_____________________________________________________
------------------------------
Number of Objects : 7703
Number of Columns : 42
_____________________________________________________


In [53]:
import joblib
from tqdm import tqdm
from utilities import cv , get_score
from sklearn.ensemble import GradientBoostingClassifier , RandomForestClassifier

# {'max_depth': 30, 'n_estimators': 400} for MODE imputation
# {'max_depth': 180, 'n_estimators': 400} for RF imputation
for i in (range(5)):
    print('-------------------------------------')
    print(f'DOING {i}th Iteration')
    rf = RandomForestClassifier(n_estimators=400 , max_depth=30 , n_jobs=-1 , random_state=np.random.randint(0,999999))
    res_final  = cv(
        {'data' : data , 'name' : f'data_10iter_rfimp_tuned'},   
        {'model' : rf , 'name' :'GB'} , ## CHANGE HERE########################################################
        k = 5 , return_dict = ret_dict, save_df= False , multiprocessing = False)
    score = get_score([res_final['prob_table']])
    joblib.dump(score , f'result/classifier_scores/rf_mode_shuffle_no_smote{i}.pkl')
    joblib.dump(res_final , f'result/prob_table/rf_mode_shuffle_no_smote{i}.pkl' , compress=3)

-------------------------------------
DOING 0th Iteration
Doing 5 fold cross-validation , with reshuffling


100%|██████████| 5/5 [00:30<00:00,  6.05s/it]


[RESULT] Overall Accuracy : 0.91
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 1th Iteration
Doing 5 fold cross-validation , with reshuffling


100%|██████████| 5/5 [00:30<00:00,  6.04s/it]


[RESULT] Overall Accuracy : 0.91
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 2th Iteration
Doing 5 fold cross-validation , with reshuffling


...

[RESULT] Overall Accuracy : 0.91
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 3th Iteration
Doing 5 fold cross-validation , with reshuffling


100%|██████████| 5/5 [00:38<00:00,  7.60s/it]


[RESULT] Overall Accuracy : 0.90
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 4th Iteration
Doing 5 fold cross-validation , with reshuffling


100%|██████████| 5/5 [00:36<00:00,  7.26s/it]


[RESULT] Overall Accuracy : 0.91
[INFO] Validation probability table is available as ['prob_table'] in response


In [43]:
get_score([res_final['prob_table']] , sc_average= 'macro')

{'classes': array(['AGN', 'CV', 'HMXB', 'LMXB', 'PULSAR', 'STAR', 'ULX', 'YSO'],
       dtype=object),
 'num_src':         pred_class
 STAR          2728
 AGN           2311
 YSO           1164
 HMXB           797
 ULX            248
 CV             188
 PULSAR         138
 LMXB           129,
 'avg_scores': {'balanced_accuracy': 0.7581580371027818,
  'accuracy': 0.8993898481111255,
  'precision': 0.7330131695400041,
  'recall': 0.7581580371027818,
  'f1': 0.7432281277109776},
 'mcc': 0.8649405367150832,
 'class_scores':         recall_score  precision_score  f1_score
 class                                          
 AGN         0.932777         0.966681  0.949426
 CV          0.542169         0.478723  0.508475
 HMXB        0.839572         0.787955  0.812945
 LMXB        0.804196         0.891473  0.845588
 PULSAR      0.465347         0.340580  0.393305
 STAR        0.939068         0.960411  0.949619
 ULX         0.616114         0.524194  0.566449
 YSO         0.926023         0.9

In [5]:
import joblib
from tqdm import tqdm
from utilities import cv , get_score
from sklearn.ensemble import GradientBoostingClassifier , RandomForestClassifier

# {'max_depth': 30, 'n_estimators': 400} for MODE imputation
# {'max_depth': 180, 'n_estimators': 400} for RF imputation
for i in (range(5,15)):
    print('-------------------------------------')
    print(f'DOING {i}th Iteration')
    # rf = RandomForestClassifier(n_estimators=400 , max_depth=30 , n_jobs=-1 , random_state=np.random.randint(0,999999))
    gb = GradientBoostingClassifier(random_state=np.random.randint(0,999999))
    res_final  = cv(
        {'data' : data , 'name' : f'data_10iter_rfimp_tuned'},   
        {'model' : gb, 'name' :'GB'} , ## CHANGE HERE########################################################
        k = 20 , return_dict = ret_dict, save_df= False , multiprocessing = True)
    score = get_score([res_final['prob_table']])
    joblib.dump(score , f'result/classifier_scores/gb_mode_{i}.pkl')
    joblib.dump(res_final, f'result/prob_table/gb_mode_{i}.pkl' , compress=3)

-------------------------------------
DOING 5th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.89
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 6th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.89
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 7th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.89
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 8th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.89
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 9th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.89
[INFO] Validation probability table is available as ['prob_table'] in resp

In [5]:
def score_mean(file , num=5):
    labels = ['AGN' ,'STAR' , 'YSO' , 'HMXB' , 'LMXB' , 'ULX' , 'CV' , 'PULSAR']
    scores = ['precision' , 'recall' , 'f1_score']
    all_feat = {
        'AGN' : {
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        } ,
        'YSO' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'STAR' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'HMXB' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'LMXB' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        } , 
        'ULX' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'CV':{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'PULSAR' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        }
    }
    for l in labels:
        for i in range(num):
            model = joblib.load(f'result/classifier_scores/{file}{i}.pkl')
            all_feat[l]['precision'].append(model['class_scores']['precision_score'][l])
            all_feat[l]['recall'].append(model['class_scores']['recall_score'][l])
            all_feat[l]['f1_score'].append(model['class_scores']['f1_score'][l])
    all_mean = {}
    for l in labels:
        all_mean[l] = {}
        for s in scores:
            all_mean[l][s] = np.mean(all_feat[l][s])*100
    all_std = {}
    for l in labels:
        all_std[l] = {}
        for s in scores:
            all_std[l][s] = np.std(all_feat[l][s])*100
    mean_df = pd.DataFrame(all_mean)
    std_df = pd.DataFrame(all_std)
    return mean_df , std_df

In [56]:
import joblib
score = score_mean('rf_mode_shuffle_no_smote' , 5)
display(score[0].T)
display(score[1].T)

,precision,recall,f1_score
AGN,93.846294,96.400835,95.106287
STAR,93.763672,95.376344,94.562981
YSO,89.887637,93.907746,91.853481
HMXB,79.955907,84.679144,82.249191
LMXB,98.703355,73.846154,84.480048
ULX,67.476817,50.995261,58.070325
CV,65.997559,40.000000,49.798009
PULSAR,48.097592,21.386139,29.518202


,precision,recall,f1_score
AGN,0.143839,0.138229,0.088331
STAR,0.193604,0.090675,0.093848
YSO,0.358337,0.226952,0.264992
HMXB,0.338463,0.300133,0.236744
LMXB,0.927936,0.559441,0.379068
ULX,2.547608,1.630773,1.694479
CV,1.625360,1.729241,1.666982
PULSAR,1.404856,2.222767,2.092001


In [51]:
import joblib
score = score_mean('rf_mode_shuffle' , 5)
display(score[0].T)
display(score[1].T)

,precision,recall,f1_score
AGN,96.468041,93.043841,94.724546
STAR,95.834160,93.992832,94.904406
YSO,91.541752,92.306353,91.922294
HMXB,77.795608,83.074866,80.346569
LMXB,85.648352,79.300699,82.350693
ULX,52.821663,62.654028,57.310131
CV,48.286997,53.614458,50.806820
PULSAR,34.415816,44.950495,38.965167


,precision,recall,f1_score
AGN,0.156732,0.315344,0.143567
STAR,0.093554,0.165962,0.057675
YSO,0.176698,0.224266,0.158559
HMXB,0.428361,0.467723,0.189790
LMXB,0.493476,0.948578,0.690146
ULX,1.540475,1.386610,1.309886
CV,1.775149,1.475596,1.586464
PULSAR,0.717735,1.481845,0.604398


In [32]:
import joblib
score = score_mean('gb_mode' , 15)
display(score[0].T)
display(score[1].T)

,precision,recall,f1_score
AGN,97.839297,89.606124,93.541487
STAR,97.102954,91.376344,94.152494
YSO,91.187763,93.652451,92.403281
HMXB,83.384978,87.397504,85.340758
LMXB,77.465074,80.792541,79.065962
ULX,47.094498,75.039494,57.864335
CV,42.381689,60.080321,49.692752
PULSAR,28.346037,55.775578,37.582852


,precision,recall,f1_score
AGN,0.133853,0.336962,0.184300
STAR,0.152102,0.198831,0.131253
YSO,0.283979,0.276075,0.205416
HMXB,0.770292,0.661221,0.486182
LMXB,2.772887,0.879625,1.491830
ULX,1.113081,1.089933,1.051159
CV,1.022605,1.608432,1.048964
PULSAR,1.160929,1.831609,1.365245


In [33]:
import joblib
score = score_mean('rf_mode' , 15)
display(score[0].T)
display(score[1].T)

,precision,recall,f1_score
AGN,96.717311,93.324983,94.990783
STAR,95.883272,94.107527,94.987019
YSO,91.611718,92.445605,92.026374
HMXB,79.155932,83.413547,81.227787
LMXB,84.789202,80.419580,82.537216
ULX,52.411316,63.507109,57.422471
CV,49.141564,53.815261,51.360877
PULSAR,35.836795,47.062706,40.681707


,precision,recall,f1_score
AGN,0.141711,0.146924,0.112358
STAR,0.122894,0.144440,0.101347
YSO,0.238386,0.296762,0.188598
HMXB,0.655514,0.542429,0.520175
LMXB,1.348869,1.052829,0.818290
ULX,1.029965,1.080735,0.898662
CV,1.065667,1.177741,0.808506
PULSAR,1.877759,2.628687,2.122555


# Average scores

In [121]:
score = {
    'accuracy' : [],
    'precision' : [], 
    'recall' : [],
    'f1_score' : [] , 
    'mcc' : []
}
for i in range(5):
    temp = joblib.load(f'result/prob_table/rf_mode_{i}.pkl')
    score['accuracy'].append(temp['avg_scores']['accuracy']*100)
    score['precision'].append(temp['avg_scores']['precision']*100)
    score['recall'].append(temp['avg_scores']['recall']*100)
    score['f1_score'].append(temp['avg_scores']['f1']*100)
    score['mcc'].append(temp['mcc'])

In [122]:
print('accuracy :' , np.mean(score['accuracy']) , np.std(score['accuracy']))
print('precision :' , np.mean(score['precision']) , np.std(score['precision']))
print('recall :' , np.mean(score['recall']) , np.std(score['recall']))
print('f1 :' , np.mean(score['f1_score']) , np.std(score['f1_score']))
print('mcc :' , np.mean(score['mcc']) , np.std(score['mcc']))


accuracy : 90.92820978839413 0.05408974855511051
precision : 90.32620716634699 0.06435587305543901
recall : 90.92820978839413 0.05408974855511051
f1 : 90.41222662077027 0.05854608335311202
mcc : 0.8764900029262218 0.0007293516152392633


In [124]:
prob_table = joblib.load('result/classifier_scores/rf_mode_0.pkl')
prob_table

{'classes': array(['AGN', 'CV', 'HMXB', 'LMXB', 'PULSAR', 'STAR', 'ULX', 'YSO'],
       dtype=object),
 'num_src':         pred_class
 STAR          2829
 AGN           2467
 YSO           1207
 HMXB           787
 ULX            151
 LMXB           109
 CV             101
 PULSAR          52,
 'avg_scores': {'balanced_accuracy': 0.6989552185297148,
  'accuracy': 0.9086070362196547,
  'precision': 0.9022987594200812,
  'recall': 0.9086070362196547,
  'f1': 0.9033687333540134},
 'mcc': 0.8755667280539116,
 'class_scores':         recall_score  precision_score  f1_score
 class                                          
 AGN         0.967850         0.939603  0.953517
 CV          0.403614         0.663366  0.501873
 HMXB        0.852941         0.810673  0.831270
 LMXB        0.755245         0.990826  0.857143
 PULSAR      0.227723         0.442308  0.300654
 STAR        0.952688         0.939555  0.946076
 ULX         0.488152         0.682119  0.569061
 YSO         0.943429         0.8

In [134]:
ret_dict= {
    'clf': True,
 'prob_table': True,
 'acc': True,
 'pr_score': True,
 'precision': True,
 'recall': True , 
 'roc_auc_score' : True
 }
rf = RandomForestClassifier(n_estimators=400 , max_depth=30 , n_jobs=-1 , random_state=np.random.randint(0,999999))
res_final  = cv(
    {'data' : data , 'name' : f'data_10iter_rfimp_tuned'},   
    {'model' : rf , 'name' :'GB'} , ## CHANGE HERE########################################################
    k = 20 , return_dict = ret_dict, save_df= False , multiprocessing = False)
score = get_score([res_final['prob_table']])
joblib.dump(res_final , 'result/all_result/rf.pkl')

Doing 20 fold cross-validation


100%|██████████| 20/20 [02:21<00:00,  7.10s/it]


[RESULT] Overall Accuracy : 0.91
[INFO] Validation probability table is available as ['prob_table'] in response
>>> Training the final classifier RandomForestClassifier(max_depth=30, n_estimators=400, n_jobs=-1,
                       random_state=997096)
[DONE] Classifier is trained | acces via ['clf'] in the response


['result/all_result/rf.pkl']

In [133]:
ret_dict= {
    'clf': True,
 'prob_table': True,
 'acc': True,
 'pr_score': True,
 'precision': True,
 'recall': True , 
 'roc_auc_score' : True
 }
rf = RandomForestClassifier(n_estimators=400 , max_depth=30 , n_jobs=-1 , random_state=np.random.randint(0,999999))
gb = GradientBoostingClassifier()
res_final  = cv(
    {'data' : data , 'name' : f'data_10iter_rfimp_tuned'},   
    {'model' : gb , 'name' :'GB'} , ## CHANGE HERE########################################################
    k = 20 , return_dict = ret_dict, save_df= False , multiprocessing = False)
score = get_score([res_final['prob_table']])
joblib.dump(res_final , 'result/all_result/gb.pkl')

Doing 20 fold cross-validation


100%|██████████| 20/20 [20:42<00:00, 62.11s/it]


[RESULT] Overall Accuracy : 0.92
[INFO] Validation probability table is available as ['prob_table'] in response
>>> Training the final classifier GradientBoostingClassifier()
[DONE] Classifier is trained | acces via ['clf'] in the response


['result/all_result/gb.pkl']

# CMP distribution

In [ ]:
rf = joblib.load('result/all_result/gb.pkl')